In [1]:
import json
from urllib.parse import urlencode
from urllib.request import urlretrieve
import pandas as pd
import numpy as np

In [2]:
def get_nba_data(endpt, params, return_url=False):

    ## endpt: https://github.com/seemethere/nba_py/wiki/stats.nba.com-Endpoint-Documentation
    ## params: dictionary of parameters: i.e., {'LeagueID':'00'}

    useragent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""

    dataurl = "\"" + "http://stats.nba.com/stats/" + endpt + "?" + urlencode(params) + "\""
    
    # for debugging: just return the url
    if return_url:
        return(dataurl)
    
    jsonstr = !wget -q -O - --user-agent={useragent} {dataurl}
    
    data = json.loads(jsonstr[0])
    
    h = data['resultSets'][0]['headers']
    d = data['resultSets'][0]['rowSet']
    
    return(pd.DataFrame(d, columns=h))

Start by exploring combine data

In [3]:
year = "2012-13"
params = {"LeagueID":"00",
          "SeasonYear":year}
endpt = "draftcombinedrillresults"
df1 = get_nba_data(endpt=endpt,params=params)

In [4]:
endpt = "draftcombineplayeranthro"
df2 = get_nba_data(endpt=endpt,params=params)

In [5]:
year = "2013-14"
params = {"LeagueID":"00",
          "SeasonYear":year}
endpt = "draftcombinenonstationaryshooting"
df3 = get_nba_data(endpt=endpt,params=params)

In [6]:
df = pd.merge(df1,df2,on="PLAYER_ID")

Not looking like we'll have the shooting data very constitently. Lets merge two we have consitently and compile it over several years

In [7]:
dfCombineFull = pd.DataFrame(columns=df.columns)
for i in range(0,17):
    year="20"+format(i,'02d')+"-"+format(i+1,'02d')
    params.update({"SeasonYear":year})
    df1 = get_nba_data(endpt="draftcombinedrillresults",params=params)
    df2 = get_nba_data(endpt="draftcombineplayeranthro",params=params)
    dfMerged = pd.merge(df1,df2,on="PLAYER_ID")
    dfCombineFull = pd.concat([dfCombineFull,dfMerged])

In [37]:
dfCombineFull = dfCombineFull.set_index('PLAYER_ID')
dfCombineFull

,TEMP_PLAYER_ID_x,FIRST_NAME_x,LAST_NAME_x,PLAYER_NAME_x,POSITION_x,STANDING_VERTICAL_LEAP,MAX_VERTICAL_LEAP,LANE_AGILITY_TIME,MODIFIED_LANE_AGILITY_TIME,THREE_QUARTER_SPRINT,...,HEIGHT_W_SHOES,HEIGHT_W_SHOES_FT_IN,WEIGHT,WINGSPAN,WINGSPAN_FT_IN,STANDING_REACH,STANDING_REACH_FT_IN,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH
PLAYER_ID,,,,,,,,,,,,,,,,,,,,,
2124,None,Malik,Allen,Malik Allen,PF-C,25.5,29.0,11.83,None,3.38,...,None,None,271,86.50,7' 2.5'',109.0,9' 1'',None,None,None
12019,None,Harold,Arceneaux,Harold Arceneaux,SG-SF,NaN,29.0,13.80,None,NaN,...,None,None,219,80.50,6' 8.5'',103.0,8' 7'',None,None,None
12020,None,Lamont,Barnes,Lamont Barnes,PF-C,28.0,29.5,12.30,None,3.40,...,None,None,235.5,87.50,7' 3.5'',108.0,9' 0'',None,None,None
12131,None,Mario,Bland,Mario Bland,PF,27.0,31.0,13.04,None,3.47,...,None,None,287,84.00,7' 0'',103.0,8' 7'',None,None,None
2056,None,Primoz,Brezec,Primoz Brezec,C,26.0,29.5,11.53,None,3.55,...,None,None,243,86.00,7' 2'',110.0,9' 2'',None,None,None
2081,None,Ernest,Brown,Ernest Brown,C,NaN,NaN,NaN,None,NaN,...,None,None,222.5,85.25,7' 1.25'',109.5,9' 1.5'',None,None,None
2049,None,Speedy,Claxton,Speedy Claxton,PG,36.0,42.5,10.48,None,3.06,...,None,None,166,72.00,6' 0'',94.5,7' 10.5'',None,None,None
12132,None,Eric,Coley,Eric Coley,SG-SF,35.0,38.0,11.40,None,3.15,...,None,None,205,83.00,6' 11'',102.0,8' 6'',None,None,None
12133,None,Ed,Cota,Ed Cota,PG,26.5,33.5,10.98,None,3.24,...,None,None,189,76.00,6' 4'',96.5,8' 0.5'',None,None,None


In [9]:
endpt = 'playerdashptshotdefend'
params = {'DateFrom':'','DateTo':'','GameSegment':'',
          'LastNGames':'0','LeagueID':'00','Location':'',
          'Month':'0','OpponentTeamID':'0','Outcome':'',
          'Period':'0','PlayerID':'0','Season':year,
          'SeasonSemgent':'','SeasonType':'Regular Season',
          'TeamID':'0','VsConference':'','VsDivision':'',
          'PerMode':'Totals','SeasonSegment':''
         }
dfDefense = get_nba_data(endpt=endpt,params=params)
dfDefense[dfDefense.DEFENSE_CATEGORY=="Less Than 6 Ft"]


,CLOSE_DEF_PERSON_ID,GP,G,DEFENSE_CATEGORY,FREQ,D_FGM,D_FGA,D_FG_PCT,NORMAL_FG_PCT,PCT_PLUSMINUS
1452,101139,75,51,Less Than 6 Ft,0.201,60.0,92.0,0.652,0.622,0.030
1453,202703,69,66,Less Than 6 Ft,0.357,122.0,228.0,0.535,0.600,-0.065
1454,2749,74,56,Less Than 6 Ft,0.145,70.0,94.0,0.745,0.617,0.128
1455,203925,51,46,Less Than 6 Ft,0.299,83.0,123.0,0.675,0.618,0.057
1456,2730,74,72,Less Than 6 Ft,0.369,143.0,270.0,0.530,0.608,-0.079
1457,203148,39,18,Less Than 6 Ft,0.205,19.0,26.0,0.731,0.587,0.144
1458,201577,81,80,Less Than 6 Ft,0.498,328.0,618.0,0.531,0.612,-0.081
1459,202389,54,52,Less Than 6 Ft,0.406,120.0,215.0,0.558,0.603,-0.045
1460,2738,76,63,Less Than 6 Ft,0.281,101.0,162.0,0.623,0.606,0.018
1461,201936,40,33,Less Than 6 Ft,0.308,51.0,76.0,0.671,0.597,0.074


In [48]:
dfDefense = dfDefense[dfDefense.DEFENSE_CATEGORY=="Less Than 6 Ft"]
dfDefense = dfDefense.set_index("CLOSE_DEF_PERSON_ID")

In [51]:
dfDefense_PCT_Change = pd.DataFrame(dfDefense.PCT_PLUSMINUS)
dfDefense_PCT_Change

,PCT_PLUSMINUS
CLOSE_DEF_PERSON_ID,
101139,0.030
202703,-0.065
2749,0.128
203925,0.057
2730,-0.079
203148,0.144
201577,-0.081
202389,-0.045
2738,0.018


In [53]:
dfCombineFull = pd.merge(dfCombineFull,dfDefense_PCT_Change,left_index=True,right_index=True)

In [54]:
dfCombineFull.groupby('WINGSPAN')['PCT_PLUSMINUS'].mean()

WINGSPAN
70.00    0.357000
72.75    0.117000
73.75    0.139000
74.00    0.139000
74.25    0.270000
74.50    0.128000
74.75    0.095000
75.00    0.285000
75.25    0.129000
75.50    0.118500
76.00    0.136250
76.25    0.079000
76.50    0.096333
77.00   -0.050000
77.50    0.071750
77.75    0.100000
78.00    0.118000
78.25    0.020667
78.50    0.049000
78.75    0.103000
79.00   -0.082600
79.25    0.058875
79.50    0.133667
79.75    0.035800
80.00    0.056750
80.25    0.123000
80.50    0.429000
80.75    0.046000
81.00    0.043444
81.25    0.034750
           ...   
83.25    0.053125
83.50    0.033750
83.75   -0.083000
84.00   -0.000800
84.25   -0.009500
84.50    0.008000
84.75   -0.054667
85.00   -0.012667
85.25   -0.052714
85.50    0.018333
85.75    0.005000
86.00    0.010500
86.25    0.032500
86.50    0.009167
86.75   -0.005000
87.00   -0.020400
87.25   -0.016000
87.50   -0.028500
87.75   -0.068000
88.00   -0.052750
88.25   -0.061000
88.50   -0.005000
88.75   -0.070750
89.00   -0.005000
8